# Implement beer game described by John Sterman 1989

In [7]:
def calc_stock_t(stock_tm1, acquisition_rate_tm1, loss_rate_tm1, dt):
    """Calculate stock at time t
    
        S(t) = S(t-1) + (A(t-1) - L(t-1)) * dt
    
    This is a discrete version of Eqn. 1 in Sterman (1989).
    """
    return stock_tm1 + (acquisition_rate_tm1 - loss_rate_tm1) * dt

assert calc_stock_t(10, 4, 3, 1) == 11
assert calc_stock_t(10, 4, 3, 0.5) == 10.5

In [9]:
def calc_supply_line_t(supply_line_tm1, order_rate_tm1, acquisition_rate_tm1, dt):
    """Calculate supply line at time t
    
        SL(t) = SL(t-1) + (O(t-1) - A(t-1)) * dt
    
    This is a discrete version of Eqn. 2 in Sterman (1989).
    """
    return supply_line_tm1 + (order_rate_tm1 - acquisition_rate_tm1) * dt

assert calc_supply_line_t(10, 4, 3, 1) == 11
assert calc_supply_line_t(10, 4, 3, 0.5) == 10.5

In [11]:
# Managers are assumed to choose orders so as to: (1) replace 
# expected losses from stock; (2) reduce the discrepancy 
# between the desired and actual stock; and (3) maintain
# an adequate supply line of unfilled orders.

# Orders must be non-negative
# orders_t = max(0, indicated_orders)

In [14]:
# Indicated order rate is based on the anchoring and 
# adjustment heuristic (Tversky and Kahneman 1974).
# Anchoring and adjustment is a common strategy in which 
# an unknown quantity is estimated by first recalling a
# known reference point (the anchor) and then adjusting
# for the effects of other factors which may be less
# salient or whose effects are obscure, requiring the
# subject to estimate these effects by what Kahneman and
# Tversky (1982) call 'mental simulation.' Here, the
# anchor is the expected loss rate L. Adjustments are then
# made to correct discrepancies between the desired and
# actual stock (AS), and between the desired and actual 
# supply line (ASL).

def calc_order_rate_t(expected_loss_rate_t, adjustment_stock_t, adjustment_supply_line_t):
    return expected_loss_rate_t + adjustment_stock_t + adjustment_supply_line_t